In [24]:
import os
import json
import pandas as pd
import traceback


In [25]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain_community.callbacks import get_openai_callback
import PyPDF2
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
SECRET_KEY=os.getenv("OPENAI_API_KEY")

In [27]:
llm = ChatOpenAI(
    openai_api_key=SECRET_KEY,
    model="gpt-3.5-turbo",
    temperature=0.5,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [28]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [29]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [30]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [31]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [32]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [33]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [40]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"], 
    verbose=True,
    )

In [49]:
file_path=r"C:\Users\MSI TM\MCQ_Generator\data.txt"
with open(file_path, 'r') as file:
    TEXT=file.read()
    #print(TEXT)

In [50]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [53]:
NUMBER=5
SUBJECT="generative ai"
TONE="simple"

In [55]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\MSI TM\AppData\Local\Temp\ipykernel_22756\1250213335.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term “democratizing innovation” was coined by MIT’s Eric von Hippel, who, since the mid-1970s, has been researching and writing about the potential for users of products and services to develop what they need themselves rather than simply relying on companies to do so. In the past two decades or so, the notion of deeply involving users in the innovation process has taken off, and today companies use crowdsourcing and innovation contests to generate a multitude of new ideas. However, many enterprises struggle to capitalize on these contributions because of four challenges.

First, efforts to democratize innovation may result in evaluation overload. Crowdsourcing, for instance, may produce a flood of ideas, many of which end up being dumped or disregarded because companies have no efficient way to evaluate them or merge incomplete or minor ideas that could prove potent in combi

In [56]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:9693
Prompt Tokens:8898
Completion Tokens:795
Total Cost:0.0056415


In [57]:
response

{'text': 'The term “democratizing innovation” was coined by MIT’s Eric von Hippel, who, since the mid-1970s, has been researching and writing about the potential for users of products and services to develop what they need themselves rather than simply relying on companies to do so. In the past two decades or so, the notion of deeply involving users in the innovation process has taken off, and today companies use crowdsourcing and innovation contests to generate a multitude of new ideas. However, many enterprises struggle to capitalize on these contributions because of four challenges.\n\nFirst, efforts to democratize innovation may result in evaluation overload. Crowdsourcing, for instance, may produce a flood of ideas, many of which end up being dumped or disregarded because companies have no efficient way to evaluate them or merge incomplete or minor ideas that could prove potent in combination.\n\nSecond, companies may fall prey to the curse of expertise. Domain experts who are bes

In [61]:
quiz=response.get("quiz")

In [63]:
quiz=json.loads(quiz)

In [64]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [65]:
quiz_table_data

[{'MCQ': "What term was coined by MIT's Eric von Hippel to describe the potential for users to develop what they need themselves?",
  'Choices': 'a: Innovative Revolution | b: Innovation Overload | c: Democratizing Innovation | d: User Dependency',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is a challenge faced by companies in democratizing innovation according to the text?',
  'Choices': 'a: Evaluation Underload | b: Curse of Expertise | c: Lack of Ideas | d: Forest Clarity',
  'Correct': 'b'},
 {'MCQ': 'How can generative AI support divergent thinking?',
  'Choices': 'a: By limiting associations among concepts | b: By restricting idea generation | c: By making associations among remote concepts | d: By discouraging creativity',
  'Correct': 'c'},
 {'MCQ': 'What is one way generative AI can assist in idea evaluation?',
  'Choices': 'a: By replacing human evaluators | b: By reducing the number of ideas generated | c: By increasing the specificity of ideas | d: By discouraging 

In [66]:
quiz=pd.DataFrame(quiz_table_data)

In [67]:
quiz.to_csv("generativeai.csv",index=False)

In [68]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'10_10_2024_14_39_58'